# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [27]:
# To see log messages
from xopt import output_notebook
output_notebook('DEBUG')

# Import the class
from xopt import Xopt
from botorch.test_functions.multi_fidelity import AugmentedHartmann
import os
SMOKE_TEST = os.environ.get('SMOKE_TEST')


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [28]:
# Make a proper input file. 
import yaml
YAML = """
xopt: {output_path: null}

algorithm:
  name: multi_fidelity
  options:  
      batch_size: 4
      budget: 200
      fixed_cost: 5.0
      n_initial_samples: 16

simulation: 
  name: test_multi_fidelity
  evaluate: xopt.tests.test_functions.multi_fidelity.evaluate

vocs:
  description: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [29]:
if SMOKE_TEST:
    config['algorithm']['options']['budget'] = 3
    config['algorithm']['options']['processes'] = 1
    config['algorithm']['options']['generator_options']['num_restarts'] = 2
    config['algorithm']['options']['generator_options']['raw_samples'] = 2
    config['algorithm']['options']['generator_options']['base_acq'] = None

X = Xopt(config)
X

Loading config from dict.
`description` keyword no longer allowed in vocs config, removing
`evaluate key in evaluate no longer allowed, replacing with `function` keyword
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.


C:\Users\Ryan Roussel\Documents\GitHub\Xopt\xopt\legacy.py:40: UserWarning: `simulation` keyword no longer allowed in evaluate config, moving to `evaluate`
  warnings.warn(



            Xopt 
________________________________           
Version: 0.5.0+16.g28935f4.dirty
Configured: True
Config as YAML:
xopt: {output_path: null}
algorithm:
  name: multi_fidelity
  options: {batch_size: 4, budget: 200, fixed_cost: 5.0, n_initial_samples: 16, output_path: .,
    restart_file: null, num_restarts: 20, raw_samples: 1024, num_fantasies: 128}
  type: batched
  function: null
vocs:
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
  constraints: {}
evaluate:
  name: test_multi_fidelity
  function: xopt.tests.test_functions.multi_fidelity.evaluate
  executor: null
  options: {extra_option: abc}

# Run BayesOpt

In [30]:
# Change max generations
X.run()

Starting at time 2021-10-21T15:43:52-05:00
Generating and submitting initial samples
generating samples
generated 4 samples in 21.59 seconds
samples
         x1        x2        x3        x4        x5        x6      cost
0  0.328708  0.995807  0.945844  0.549720  0.937641  0.004442  0.055705
1  0.323865  0.010545  0.449040  0.457163  0.316815  0.822545  0.130918
2  0.331510  0.855107  0.923964  0.725311  0.954624  0.006981  0.066421
3  0.242447  0.815054  0.897403  0.569898  0.843244  0.002512  0.063891
collecting results
saving data to file
generating samples
generated 4 samples in 18.6 seconds
samples
         x1        x2        x3        x4        x5        x6      cost
0  0.428529  0.945932  0.759510  0.478149  1.000000  0.000000  0.853596
1  0.435890  1.000000  1.000000  0.566096  0.774466  0.132344  0.795087
2  0.527218  0.824328  0.820093  0.660372  0.803974  0.000000  0.000000
3  0.670458  0.019290  0.474177  0.135217  0.000000  0.842108  0.000000
collecting results
saving dat

,x1,x2,x3,x4,x5,x6,cost,a,y1,status,x1_t,x2_t,x3_t,x4_t,x5_t,x6_t,cost_t,y1_t
0,0.691244,0.615999,0.871892,0.406190,0.929676,0.203446,0.258304,dummy_constant,-0.216302,done,0.691244,0.615999,0.871892,0.406190,0.929676,0.203446,0.258304,0.365187
1,0.070838,0.126777,0.804781,0.633564,0.129536,0.714260,0.273388,dummy_constant,-0.541040,done,0.070838,0.126777,0.804781,0.633564,0.129536,0.714260,0.273388,-0.077505
2,0.145854,0.156026,0.297237,0.247525,0.076005,0.126591,0.498973,dummy_constant,-0.137570,done,0.145854,0.156026,0.297237,0.247525,0.076005,0.126591,0.498973,0.472516
3,0.891793,0.493741,0.035277,0.701003,0.329419,0.407734,0.397707,dummy_constant,-0.050089,done,0.891793,0.493741,0.035277,0.701003,0.329419,0.407734,0.397707,0.591772
4,0.937207,0.892452,0.358672,0.874244,0.982042,0.767259,0.508448,dummy_constant,-0.000018,done,0.937207,0.892452,0.358672,0.874244,0.982042,0.767259,0.508448,0.660031
5,0.603009,0.086291,0.506407,0.289901,0.109287,0.762377,0.948699,dummy_constant,-0.974108,done,0.603009,0.086291,0.506407,0.289901,0.109287,0.762377,0.948699,-0.667874
6,0.078693,0.292805,0.925308,0.707332,0.068799,0.724938,0.198843,dummy_constant,-0.406239,done,0.078693,0.292805,0.925308,0.707332,0.068799,0.724938,0.198843,0.106260
7,0.371620,0.845505,0.905252,0.585629,0.890036,0.034826,0.053120,dummy_constant,-2.918608,done,0.371620,0.845505,0.905252,0.585629,0.890036,0.034826,0.053120,-3.318665
8,0.201152,0.225770,0.969912,0.275714,0.945538,0.573256,0.370739,dummy_constant,-0.039041,done,0.201152,0.225770,0.969912,0.275714,0.945538,0.573256,0.370739,0.606834
9,0.975543,0.690233,0.346602,0.621266,0.116104,0.126608,0.156477,dummy_constant,-0.011705,done,0.975543,0.690233,0.346602,0.621266,0.116104,0.126608,0.156477,0.644098


### Get highest fidelity global optimum

NOTE: the correct global minimum is -3.32237


In [31]:
import torch
model = X.generator._create_model(X.algorithm.data)
print(model.train_targets)

## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = X.generator.get_recommendation(X.algorithm.data)
print(rec)
problem = AugmentedHartmann(negate=False)
problem(torch.tensor(rec.to_numpy())) ## NOTE: the correct global minimum is -3.32237

tensor([-0.3652,  0.0775, -0.4725, -0.5918, -0.6600,  0.6679, -0.1063,  3.3187,
        -0.6068, -0.6441, -0.5883, -0.6054,  0.0780,  0.7917,  0.1766, -0.4700,
         0.3237, -0.9069,  1.2670, -0.6838,  0.2658,  0.6777,  0.5344, -1.4780,
        -1.1258,  1.2101,  0.3216, -0.4060, -0.7657,  1.0210,  0.6904, -0.9458,
        -0.8309,  0.6536, -0.8774,  1.0548, -0.4565,  1.4756, -0.2918, -0.7274,
         0.3773,  0.7066, -1.4829,  0.3990], dtype=torch.float64)
         x1        x2        x3        x4        x5        x6  cost
0  0.374909  0.886411  0.899471  0.580135  0.824566  0.033005   1.0


tensor([-2.9928], dtype=torch.float64)

In [32]:
# Cleanup
!rm results.json

'rm' is not recognized as an internal or external command,
operable program or batch file.
